# Build a New Tokenizer

* [HuggingFace video](https://www.youtube.com/watch?v=MR8tZm5ViWU)

Tokenisation Steps: 

1. Normalisation
2. Pre-tokenisation
3. Model
4. Post-processing
5. Decoding

Build-your-own Steps:

1. Gather a corpus
2. Create a `backend_tokenizer` with HF (steps 1-4 in tokenisation)
3. Load `backend_tokenizer` in a HF transformers tokenizer

# Imports

In [2]:
from datasets import load_dataset

c:\Users\bcech\anaconda3\envs\tok\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


# Dataset

In [17]:
dataset = load_dataset(name='tiny_shakespeare', path='datasets/tiny_shakespeare', split='train')

FileNotFoundError: Couldn't find a dataset script at c:\Users\bcech\Documents\ML\tolkenizers\notebooks\datasets\tiny_shakespeare\tiny_shakespeare.py or any data file in the same directory. Couldn't find 'datasets/tiny_shakespeare' on the Hugging Face Hub either: FileNotFoundError: Dataset 'datasets/tiny_shakespeare' doesn't exist on the Hub

In [11]:
import datasets
datasets.huggingface_hub.list_datasets()

AttributeError: module 'datasets' has no attribute 'huggingface_hub'